In [3]:
from os import getenv
from dotenv import load_dotenv
from hub_downloader import Downloader

%reload_ext autoreload
%autoreload 1
%aimport hub_downloader

load_dotenv();

In [18]:
#owner = 'pandas-dev'
#repo = 'pandas'
owner = 'tomgi'
repo = 'git_stats'

github_token = getenv('GITHUB_OAUTH_TOKEN')
downloader = Downloader(owner, repo, github_token)
#downloader.delete_cache()

The maximum number of requests you are permitted to make per hour: 5000
The number of requests remaining in the current rate limit window: 4912


In [21]:
downloader.get_stargazers()

,Unnamed: 0,user,starred_at
0,0,blomma,2012-10-06
1,1,volkan,2012-10-06
2,2,agladysh,2012-10-06
3,3,rbq,2012-10-06
4,4,legzo,2012-10-06
...,...,...,...
925,925,wenkw,2020-04-10
926,926,raunovillberg,2020-04-15
927,927,sensenlinlin,2020-04-21
928,928,cooka,2020-04-22


In [23]:
downloader.get_issues()

,Unnamed: 0,id,state,created_at
0,0,603734807,open,2020-04-21
1,1,591205876,open,2020-03-31
2,2,585943927,open,2020-03-23
3,3,565215119,open,2020-02-14
4,4,489233792,open,2019-09-04
5,5,413849830,open,2019-02-24
6,6,406266057,open,2019-02-04
7,7,397206370,open,2019-01-09
8,8,396169419,open,2019-01-05
9,9,379234901,open,2018-11-09


In [24]:
total_contributions, weekly_contributions = downloader.get_contributors_statistic()

In [25]:
total_contributions

,Unnamed: 0,user,commits
0,0,weeger,1
1,1,ruraj,1
2,2,riveramj,1
3,3,nickbroon,1
4,4,markus-strenn,1
5,5,kunthar,1
6,6,kjivan,1
7,7,irevert,1
8,8,flobee,1
9,9,codebreach,1


In [26]:
weekly_contributions

,Unnamed: 0,user,week_unix_ts,date,additions,deletions,commits
0,0,weeger,1349568000,2012-10-07,0,0,0
1,1,weeger,1350172800,2012-10-14,0,0,0
2,2,weeger,1350777600,2012-10-21,0,0,0
3,3,weeger,1351382400,2012-10-28,0,0,0
4,4,weeger,1351987200,2012-11-04,0,0,0
...,...,...,...,...,...,...,...
6727,6727,tomgi,1586044800,2020-04-05,0,0,0
6728,6728,tomgi,1586649600,2020-04-12,0,0,0
6729,6729,tomgi,1587254400,2020-04-19,0,0,0
6730,6730,tomgi,1587859200,2020-04-26,0,0,0


In [28]:
downloader.get_code_frequency_statistic()

,Unnamed: 0,week_unix_ts,additions,deletions,date
0,0,1349568000,10821,-796,2012-10-07
1,1,1350172800,1800,-903,2012-10-14
2,2,1350777600,1906,-1161,2012-10-21
3,3,1351382400,10,-14,2012-10-28
4,4,1351987200,0,0,2012-11-04
...,...,...,...,...,...
391,391,1586044800,0,0,2020-04-05
392,392,1586649600,0,0,2020-04-12
393,393,1587254400,0,0,2020-04-19
394,394,1587859200,0,0,2020-04-26
